# Clustering y PCA sobre dataset de work-life balance

## Laboratorio 2022 - Tarea 4

### Grupo 27

Alexis Baladón, CI: 5.574.612-4

Ignacio Viscardi, CI: 5.066.666-2

Rafael Castelli, CI: 5.079.391-2

## Requisitos para reproducción de algoritmo

Para correr el presente notebook se deberán instalar las siguientes dependencias. Queda un script para su instalación automática.

In [ ]:
!pip install -r requirements.txt

## 2. Decisiones de diseño

La finalidad de esta sección consiste en introducir las principales decisiones de diseño tomadas para esta implementación de K-means.

### 2.1 Preprocesado

Primeramente, se elimina el nombre de la ciudad y país del dataset. Esto se hace debido a que a que ambos atributos son strings por lo que al menos que realicemos alguna tecnica como one-hot encoding no seriamos capaces de utilizarlo con el algoritmo K-means, ademas no se consideran relevantes para medir el work-life balance.
Por otra parte, también se eliminó la columna 2021 y 2022 ya que estos atributos no parecen ser relevantes para el problema en cuestión.

En la seccion 3 (Experimentación) se mencionarán, cuando se considere adecuado, otras variables que fueron eliminadas del conjunto de datos para realizar los experimentos pertinentes.

### 2.2 Metaparametros

-- explicar los metaparametros

### 2.3 Metodo de inicialización
Se consideraron 2 métodos de inicialización para los centroides del algoritmo K-means. Uno de ellos realiza una selección aleatoria de K elemento del espacio de posibles elementos (se considera que un elemento x esta en el espacio de posibles elementos si para cada coordenada su valor es menor o igual que el maximo valor, en valor absoluto, que puede tomar dicha coordenada en el dataset provisto)
elementos del dataset para que sean los centroides iniciales, mientras que el otro

### 2.4 Manejo de cluster vacios en kmeans

exlicar que nos quedamos con el de menor distancia al centroide vacio y re asignamos

## 3 Experimentos

### 3.1 Sin normalizacion


Prerequisitos:

In [ ]:
import preprocessing
import constants
import os
from test import Test


original_df = preprocessing.load_csv()
df = preprocessing.preprocess(original_df, shouldNormalize=False)
test = Test()
k_values = [x for x in range(1, constants.MAX_K_PLUS_1)]


Elbow method

In [ ]:
test.test_elbow_method(dataset=df,k_values=k_values,cpu_count=os.cpu_count())

silhouette

In [ ]:
k_values = [x for x in range(2, constants.MAX_K_PLUS_1)]
test.test_silhouette(k_values=k_values, dataset=df, cpu_count=os.cpu_count())

### 3.2 Con normalizacion (copiar y pegar todo lo anterior)

In [ ]:
import preprocessing
import constants
import os
from test import Test


original_df = preprocessing.load_csv()
df = preprocessing.preprocess(original_df, shouldNormalize=True)
test = Test()
k_values = [x for x in range(1, constants.MAX_K_PLUS_1)]
test.test_elbow_method(dataset=df,k_values=k_values,cpu_count=os.cpu_count())

## 4 Conclusiones

## 4.1 Resultados según expectativas iniciales
 Joya